In [1]:
import sys
import os
import numpy as np
import pickle
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import itertools
from matplotlib.lines import Line2D
from scipy import linalg
import qinfer
import scipy

In [17]:
results_time = 'Jul_28/14_44'

results_folder = os.path.abspath("/home/bf16951/QMD/scripts/genetic_alg_param_sweep/results/")
results_dir = os.path.join(
    results_folder, 
    results_time
)

results = pd.read_csv(
    os.path.join(results_dir, 'results.csv')
)

In [18]:
results

,Unnamed: 0,champion_f_score,config_id,crossover_method,log_file,mutation_method,mutation_probability,num_protected_elite_models,number_generations,number_possible_models,number_sites,number_terms,resources,selection_method,starting_population_size,unchanged_elite_num_generations_cutoff,relative_resources,ResourceRequirement,true_found
0,0,0.933333,3.0,one_point,/home/bf16951/QMD/scripts/genetic_alg_param_sw...,element_wise,0.1,1.0,32.0,32768.0,6.0,15.0,4352.0,roulette,16.0,3.0,0.50,medium-high,False
1,1,1.000000,1.0,one_point,/home/bf16951/QMD/scripts/genetic_alg_param_sw...,element_wise,0.1,1.0,16.0,32768.0,6.0,15.0,2176.0,roulette,16.0,3.0,0.25,low,True
2,2,1.000000,5.0,one_point,/home/bf16951/QMD/scripts/genetic_alg_param_sw...,element_wise,0.1,1.0,64.0,32768.0,6.0,15.0,8704.0,roulette,16.0,3.0,1.00,high,True
3,3,1.000000,1.0,one_point,/home/bf16951/QMD/scripts/genetic_alg_param_sw...,element_wise,0.1,1.0,16.0,32768.0,6.0,15.0,2176.0,roulette,16.0,3.0,0.25,low,True
4,4,1.000000,3.0,one_point,/home/bf16951/QMD/scripts/genetic_alg_param_sw...,element_wise,0.1,1.0,32.0,32768.0,6.0,15.0,4352.0,roulette,16.0,3.0,0.50,medium-high,True
5,5,1.000000,5.0,one_point,/home/bf16951/QMD/scripts/genetic_alg_param_sw...,element_wise,0.1,1.0,64.0,32768.0,6.0,15.0,8704.0,roulette,16.0,3.0,1.00,high,True
6,6,1.000000,1.0,one_point,/home/bf16951/QMD/scripts/genetic_alg_param_sw...,element_wise,0.1,1.0,16.0,32768.0,6.0,15.0,2176.0,roulette,16.0,3.0,0.25,low,True
7,7,1.000000,3.0,one_point,/home/bf16951/QMD/scripts/genetic_alg_param_sw...,element_wise,0.1,1.0,32.0,32768.0,6.0,15.0,4352.0,roulette,16.0,3.0,0.50,medium-high,True
8,8,1.000000,5.0,one_point,/home/bf16951/QMD/scripts/genetic_alg_param_sw...,element_wise,0.1,1.0,64.0,32768.0,6.0,15.0,8704.0,roulette,16.0,3.0,1.00,high,True
9,9,1.000000,3.0,one_point,/home/bf16951/QMD/scripts/genetic_alg_param_sw...,element_wise,0.1,1.0,32.0,32768.0,6.0,15.0,4352.0,roulette,16.0,3.0,0.50,medium-high,True


In [13]:
2**15

32768